## Yolo aproach on MiTec dataset

In [1]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch import nn
from torch.optim import Adam
from torchvision.transforms import  Compose, ToTensor, Resize, RandomHorizontalFlip, RandomRotation, ColorJitter, RandomResizedCrop, Lambda
from PIL import Image
import matplotlib.pyplot as plt
from torchsummary import summary
import torchmetrics
from lightning.pytorch.loggers import WandbLogger
import wandb
import random
from datetime import datetime
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
import os

# Model
%env CUDA_VISIBLE_DEVICES=2
#%env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:32

# HF Cache
os.environ["HF_HOME"] = "../.cache"
!echo $HF_HOME
!huggingface-cli whoami

In [ ]:
from ultralytics import YOLO
import wandb

# 1. Login (nur einmal nötig pro Session/Notebook)
wandb.login()

# 2. Optional: Initialisieren, falls du etwas anpassen willst
wandb.init(
    project="Yolo-Training",    # <-- exakt wie dein Projektname
    entity="maats",             # <-- dein Benutzername oder Teamname auf W&B
    name="YOLO11n-COCO11-run1"  # optionaler Run-Name
)

# 3. YOLO-Modell laden
model = YOLO("yolo11n.pt")

# 4. Training starten – wandb ist automatisch aktiv bei installierter lib
results = model.train(
    data="/data22/stu236894/GitRepos/TinyML-MT/Dataset/coco11.yaml",
    epochs=100,
    imgsz=400,
    project="runs/train",
    name="yolo11n-exp1",
    verbose=True,
    wandb=True  # <-- das aktiviert Logging zu deinem wandb-Run
)


New https://pypi.org/project/ultralytics/8.3.115 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.107 🚀 Python-3.10.12 torch-2.5.1+cu124 CUDA:0 (NVIDIA TITAN Xp, 12183MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/data22/stu236894/GitRepos/TinyML-MT/Dataset/coco11.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=400, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, s

E0000 00:00:1745483781.098505  339853 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745483781.105828  339853 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=61

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /data22/stu236894/GitRepos/TinyML-MT/Dataset/labels/train.cache... 4380 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4380/4380 [00:00<?, ?it/s]
val: Scanning /data22/stu236894/GitRepos/TinyML-MT/Dataset/labels/val.cache... 3600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3600/3600 [00:00<?, ?it/s]


Plotting labels to /data22/stu236894/GitRepos/TinyML-MT/runs/detect/train12/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000154, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 416 train, 416 val
Using 4 dataloader workers
Logging results to /data22/stu236894/GitRepos/TinyML-MT/runs/detect/train12
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       3600      15683      0.599      0.411      0.461      0.396

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     37/100      1.32G     0.3564     0.4341     0.8642         31        416: 100%|██████████| 274/274 [00:36<00:00,  7.44it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 113/113 [00:17<00:00,  6.62it/s]


                   all       3600      15683      0.621      0.411      0.471      0.407

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     38/100      1.32G     0.3578     0.4286     0.8666         28        416: 100%|██████████| 274/274 [00:36<00:00,  7.56it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  40%|███▉      | 45/113 [00:07<00:10,  6.42it/s]


KeyboardInterrupt: 